<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/Llama_2_GPTQ%E9%87%8F%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[参考文章：大卫](https://mp.weixin.qq.com/s/BYhJcmrE9rxyHvxS6-Ph)

In [ ]:
# 安装需要依赖
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7
!pip install -q huggingface_hub sentencepiece einops
!pip install -q optimum==1.12.0 auto-gptq==0.4.2

In [ ]:
# import 密钥，token
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

In [ ]:
# Dataset id from Hugging Face
dataset_id = "wikitext2"

In [ ]:
from optimum.gptq import GPTQQuantizer

# GPTQ quantizer
quantizer = GPTQQuantizer(bits=4, dataset=dataset_id, model_seqlen=4096)
quantizer.quant_method = "gptq"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Hugging Face model id
model_id = "chenshake/Llama-2-7b-hf"

 # bug with fast tokenizer we load the model in fp16 on purpose
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True, torch_dtype=torch.float16)

In [ ]:
import os
import json


# quantize the model
quantized_model = quantizer.quantize_model(model, tokenizer)


# save the quantize model to disk
save_folder = "quantized_llama"
quantized_model.save_pretrained(save_folder, safe_serialization=True)


# load fresh, fast tokenizer and save it to disk
tokenizer = AutoTokenizer.from_pretrained(model_id).save_pretrained(save_folder)


# save quantize_config.json for TGI
with open(os.path.join(save_folder, "quantize_config.json"), "w", encoding="utf-8") as f:
  quantizer.disable_exllama = False
  json.dump(quantizer.to_dict(), f, indent=2)

In [ ]:
# 我们希望利用 exllama 内核，因此需要更改 `config.json。
with open(os.path.join(save_folder, "config.json"), "r", encoding="utf-8") as f:
  config = json.load(f)
  config["quantization_config"]["disable_exllama"] = False
with open(os.path.join(save_folder, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)
